In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [106]:
class Signal:
    
    def __init__(self, name, data, temp=False):
        self.name = name
        self.data = data
        if not temp:
            self.peaks = self.calculate__peaks() # list des pics
            self.deletion_limit = self.deletion_band(deletion=True) # float de la limit positive
        
    
    def deletion_band(self, pourcentage=0.42, nbr_std=3, deletion=False):
        deletion_limit = self.data.iloc[:int(pourcentage*len(self.data)),:].iloc[:,1].std()*nbr_std
        
        if deletion:
            peaks = list(self.peaks)
            for peak in peaks:
                if (peak.amplitude < deletion_limit) & (peak.amplitude > - deletion_limit): 
                    self.peaks.remove(peak)
    
        return deletion_limit
    
    
    def find_close_point(self, close_point):
        '''
            Recherche du point le plus proche de 'close_point' sur la courbe
            param:
                - (close_point : DataFrame) Le point dont on cherche le plus proche sur la courbe
            return: (DataFrame) contenant l'index, l'amplitude et la position du point
        '''
        
        columns=["sample_number", "time", "amplitude"]
        points = pd.DataFrame(columns=columns)
        
        for index, row in close_point.iterrows():
            time_diff = abs(row.time - self.data.iloc[:,0])
            amplitude_diff = abs(row.amplitude - self.data.iloc[:,1])

            sum_diff = time_diff + amplitude_diff
            id_min = sum_diff.idxmin()
            
            new_df = self.data.loc[[id_min],:].reset_index()
            new_df.columns = columns
            
            points = points.append(new_df, ignore_index=True)
            
        return points
    
    
    def find_symetric_point(self, first_point, peak):
        
        points = pd.DataFrame(columns=["sample_number", "time", "amplitude"])
        
        for fp in first_point.iterrows(): 
            if fp[1].time < peak.time:
                signal_cut = Signal('Tempo_gauche', self.data[self.data.iloc[:,0] > peak.time], temp=True)
            else:
                signal_cut = Signal('Tempo_droite', self.data[self.data.iloc[:,0] < peak.time], temp=True)

            points = points.append(signal_cut.find_close_point(pd.DataFrame(fp[1]).T), ignore_index=True)
        
        return points
    
    
    def peaks_list(self, kind):
        '''
            Récupère tous les pics du signal du genre 'kind'
            param:
                - (kind : str) String contenant 'max' ou 'min' en fonction des pics que l'on souhaite récupérer.
            return: (list) contenant l'ensemble des pics du signal du genre 'kind'
        '''
        peaks_df = pd.DataFrame()
        for peak in self.peaks:
            if peak.kind == kind:
                peaks_df = peaks_df.append(peak.to_df())
            elif kind == "all":
                peaks_df = peaks_df.append(peak.to_df())
        
        return peaks_df.reset_index(drop=True)
        
    
    def calcultate__half_max(self):
        peaks_df = pd.DataFrame()
        
        for peak in self.peaks:
            peaks_df = pd.concat([peaks_df, peak.calcultate__half_max])

        return peaks_df
    
    
    def calculate__peaks(self, lookahead=30, delta=0.7):
        '''
            Calcul des pics pour chacun des signaux de la list 'signaux'
            param:
                - (signaux : DataFrame) Dataframe des signaux intéressants du poids lourds
                - (lookahead : int) Distance entre le pic actuel déterminé et le candidat suivant
                - (delta : float) Différence minimum entre un pics et le suivant
            return: (list) contenant une DataFrame des pics par signaux
        '''
        peaks_list = []
        
        peaks = peakdetect(self.data.iloc[:, 1],self.data.iloc[:, 0], lookahead=lookahead, delta=delta)
        for kind in peaks:
            if kind == peaks[0]:
                kind_name = 'max'
            else:
                kind_name = 'min'
                
            for sig in kind:
                peaks_list.append(Pic(self, sig[0], sig[1], kind_name))
        
        return peaks_list
    
    
    def show(self):
        plt.figure(figsize=(15, 10))

        ## Courbe de capteurs
        plt.plot(self.data.loc[:,'time'], self.data.loc[:, 'amplitude'])
        
        ## Zone de suppression des pics
        plt.plot([self.data.iloc[0,0], self.data.iloc[-1,0]], [self.deletion_limit, self.deletion_limit])
        plt.plot([self.data.iloc[0,0], self.data.iloc[-1,0]], [-self.deletion_limit, -self.deletion_limit])

        ## Detections des maximas et minimas
        # Pics
        plt.scatter(self.peaks_list('all')['time'], self.peaks_list('all')['amplitude'])
        # Largeur
        plt.scatter(self.peaks_list('all')['time p1'], self.peaks_list('all')['amplitude p1'])
        plt.scatter(self.peaks_list('all')['time p2'], self.peaks_list('all')['amplitude p2'])

        plt.legend(['Signal', 'Limit +', 'Limit -', 'Pics', '1er point largeur', '2nd point largeur'])
        plt.xlabel("Temps")
        plt.ylabel("Amplitude")

        plt.show()

In [98]:
class Pic:
    
    def __init__(self, signal, time, amplitude, kind):
        self.signal = signal
        self.time = time
        self.amplitude = amplitude
        self.sample_number = self.find_sample_number()
        self.fwhm = self.calculate__fwhm() # float de la lagueur à mi-hauteur du pic
        self.kind = kind
        
    
    def to_df(self):
        df = pd.DataFrame(data=[[self.time, self.amplitude, self.sample_number, self.fwhm[0].loc['time',:][0], self.fwhm[0].loc['amplitude',:][0], self.fwhm[1].loc['time',:][0], self.fwhm[1].loc['amplitude',:][0], self.fwhm[2].loc[:,'fwhm'][0] , self.kind, self.signal.name]],\
                                    columns=["time", "amplitude", "sample_number", "time p1", "amplitude p1", "time p2", "amplitude p2", "fwhm", "kind", "signal_name"])
        return df
    
    
    def calcultate__half_max(self):
        middle_time = self.time
        middle_amplitude = self.amplitude - self.amplitude/2

        return pd.DataFrame(data = [[middle_time, middle_amplitude]], columns=["time", "amplitude"])
    
    
    def find_sample_number(self):
        return self.signal.data[(self.signal.data.loc[:,'time'] == self.time) &\
                                (self.signal.data.loc[:,'amplitude'] == self.amplitude)].index[0]
    
    
    def calculate__fwhm(self):
        # Calcul mi-hauteur
        half_max = self.calcultate__half_max()
        point = self.signal.find_close_point(half_max)
        first_point = pd.DataFrame(point).T
        second_point = pd.DataFrame(self.signal.find_symetric_point(point, self)).T

        largeur = pd.DataFrame(np.transpose(np.abs(np.array(second_point.loc['time',:]) - np.array(first_point.loc['time',:]))), columns=["fwhm"])

        return (first_point, second_point, largeur)